<a href="https://colab.research.google.com/github/alessandronascimento/pyLiBELa/blob/alex-works/Colabs/GridNumba.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Setup

In [1]:
#@title Baixando pyLiBELa {display-mode: "form"}
%%capture
!pip3 install condacolab
import condacolab
condacolab.install()
! apt-get install openbabel libopenbabel-dev python-dev-is-python3 zlib1g-dev libeigen3-dev libgsl-dev libnlopt-cxx-dev libgsl-dev
! mamba install -c anaconda py-boost

In [1]:
! git init
! git remote add origin https://github.com/alessandronascimento/pyLiBELa.git
! git pull origin alex-works

Initialized empty Git repository in /content/.git/
remote: Enumerating objects: 1784, done.
remote: Counting objects: 100% (409/409), done.
remote: Compressing objects: 100% (212/212), done.
remote: Total 1784 (delta 278), reused 285 (delta 196), pack-reused 1375
Receiving objects: 100% (1784/1784), 4.68 MiB | 17.56 MiB/s, done.
Resolving deltas: 100% (1214/1214), done.
From https://github.com/alessandronascimento/pyLiBELa
 * branch            alex-works -> FETCH_HEAD
 * [new branch]      alex-works -> origin/alex-works


In [ ]:
! mv pyLiBELa/src src
! rm -rf pyLiBELa
! mkdir -p obj
! rm -f Makefile*
!wget https://raw.githubusercontent.com/alessandronascimento/pyLiBELa/main/Colabs/Makefile
!make -j4 

In [ ]:
!rm -rf obj

# Initializations

In [3]:
#@title Importando bibliotecas {display-mode: "form"}

from pyPARSER import *
from pyMol2 import *
from pyWRITER import *
from pyGrid import *
from pyCOORD_MC import *
from pyFindHB import * 
from pyEnergy2 import *
from pyGaussian import *
from pyConformer import *
from pyRAND import *
from pyMcEntropy import *
from pySA import *
from pyOptimizer import *
from pyMC import *
from pyFullSearch import *
from pyDocker import *

from tqdm import tqdm
import numpy as np
import numba as nb
from numba import jit
import pdb

In [4]:
Input = PARSER()
Input.output = "teste" #@param {type:"string"}
Input.scoring_function = 0 #@param {type:"integer"}
Input.use_grids = True #@param {type:"boolean"}
Input.grid_prefix = "McGrid" #@param {type:"string"}
Input.search_box_x = 30.0 #@param {type:"number"}
Input.search_box_y = 30.0 #@param {type:"number"}
Input.search_box_z = 30.0 #@param {type:"number"}
Input.grid_spacing = 1.0 #@param {type:"number"}
Input.x_dim = 30.0 #@param {type:"number"}
Input.y_dim = 30.0 #@param {type:"number"}
Input.z_dim = 30.0 #@param {type:"number"}
Input.write_grids = True;

# Para que funcione a funcão

Input.dielectric_model = "constant" #@param {type: "string"}

In [51]:
Writer = WRITER(Input)
Coord = COORD_MC()
HB = FindHB()
lig = Mol2(Input, "test/lig.mol2.gz")
rec = Mol2(Input, "test/rec.mol2.gz")
Energy = Energy2(Input)
com = Coord.compute_com(lig)

In [ ]:
refgrid = Grid(Input, Writer, rec, com)
py_grid = Grid(Input, Writer, rec, com)
numba_grid = Grid(Input, Writer, rec, com)

In [ ]:
for i in range(len(rec.charges)):
  
  print(rec.charges[i], '\n')

In [ ]:
refgrid.compute_grid_softcore(rec)

In [ ]:
refgrid.compute_grid_softcore_HB_omp(rec)

In [48]:
del(refgrid)

In [49]:
del(py_grid)

In [50]:
del(numba_grid)

In [ ]:
for i in range(grid.npointsy):
  print('\n')
  for j in range(grid.npointsz):
    print(f'{refgrid.rec_solv_gauss[0][i][j]:.2f}', end=' ')

# Porting work

In [ ]:
#@title compute_grid_softcore
def compute_grid_softcore(Rec, Grid, Input):
  
  # if (Input.scoring_function >= 2):
    #Grid.compute_grid_harcore(rec)
    #return

  
  for i in tqdm(range(Grid.npointsx)):
    x = (i*Grid.grid_spacing) + Grid.xbegin

    for j in range(Grid.npointsy):
      y = (j*Grid.grid_spacing) + Grid.ybegin

      for k in range(Grid.npointsz):
        z = (k*Grid.grid_spacing) + Grid.zbegin

        elec = 0
        vdwA = 0
        vdwB = 0
        solv = 0
        rec_solv = 0

        for n in range(Rec.N):
          d = Grid.distance(x, Rec.xyz[n][0], y, Rec.xyz[n][1], z, Rec.xyz[n][2])
          d2 = d**2
          d3 = d**3
          d6 = d2**3

          denom = (d3 + Input.deltaij_es3) ** (1/3)

          if (Input.dielectric_model == "constant"):
            elec += (332*Rec.charges[n]) / (Input.diel*denom)
            denom = (d6 + Input.deltaij_es6) ** (1/3)
            solv += ( ((Input.solvation_alpha * Rec.charges[n] * Rec.charges[n]) + 
                       Input.solvation_beta ) * np.exp((-denom/(2*Input.sigma**2))) / (Input.sigma**3) )
            rec_solv += (4/3) * np.pi * Rec.radii[n]**3 * np.exp((-denom/(2*Input.sigma**2))) / (Input.sigma**3)

          else:
            denom = (d6 + Input.deltaij_es6) ** (1/3)
            elec += (332 * Rec.charges[n]/(Input.diel*denom))
            solv += ( ((Input.solvation_alpha * Rec.charges[n] * Rec.charges[n]) + 
                       Input.solvation_beta ) * np.exp((-denom/(2*Input.sigma**2))) / (Input.sigma**3) )
            rec_solv += (4/3) * np.pi * Rec.radii[n]**3 * np.exp((-denom/(2*Input.sigma**2))) / (Input.sigma**3)

          denom = d6 + Input.deltaij6
          vdwA += (4096 * Rec.epsilons_sqrt[n] * Rec.radii[n]**6) / (denom**2)
          vdwB += (128 * Rec.epsilons_sqrt[n] * Rec.radii[n]**3) / (denom)

        # NOTE: assumo aqui que todos os arrays estão inicializados
        # e do tamanho correto
        Grid.elec_grid[i][j][k] = elec
        Grid.vdwA_grid[i][j][k] = vdwA
        Grid.vdwB_grid[i][j][k] = vdwB
        Grid.solv_gauss[i][j][k] = solv
        Grid.rec_solv_gauss[i][j][k] = rec_solv

  Grid.rec_si = 0
  for i in range(Rec.N):
    Grid.rec_si += (Input.solvation_alpha * Rec.charges[i] ** 2) + Input.solvation_beta



In [45]:
#@title compute_softcore_HB_omp


def compute_grid_softcore_HB_omp(rec, grid, input):
  
  #if input.scoring_function >= 2:
  #  compute_grid_hardcore_HB_omp(...)
  #  return

  HB_C12 = np.double(55332.873)
  HB_C10 = np.double(18393.199)

  for i in range(grid.npointsx):
    x = np.double(i*grid.grid_spacing + grid.xbegin) 

    for j in range(grid.npointsy):
      y = np.double(j*grid.grid_spacing + grid.ybegin) 

      for k in range(grid.npointsz):
        z = np.double(k*grid.grid_spacing + grid.zbegin) 

        elec = np.double(0.0)
        vdwA = np.double(0.0)
        vdwB = np.double(0.0) 
        solv = np.double(0.0) 
        rec_solv = np.double(0.0) 
        hb_donor = np.double(0.0) 
        hb_acceptor = np.double(0.0) 

        for a in range(rec.N):
          
          d2 = np.double(grid.distance_squared(x, rec.xyz[a][0], y, rec.xyz[a][1], z, rec.xyz[a][2]))
          d6 = np.double(np.power(d2, 3))
          
          denom = np.double(0.0)
            
          if input.dielectric_model == "constant":
            denom = np.power(d6 + input.deltaij_es6, 1/3)
            elec += (332.0*rec.charges[a])/(input.diel*denom)
            solv += ((input.solvation_alpha * rec.charges[a] * rec.charges[a]) + input.solvation_beta) * np.exp((-denom/(2 * np.power(input.sigma, 2)))) / (np.power(input.sigma, 3))
            rec_solv += (4/3) * np.pi * np.power(rec.radii[a], 3) * np.exp((-denom/(2*np.power(input.sigma, 2)))) / (np.power(input.sigma, 3))

          else:
            denom = np.power(d6 + input.deltaij_es6, 1/3)
            elec += (332.0*rec.charges[a]/(input.diel*denom))
            solv += ((input.solvation_alpha * np.power(rec.charges[a], 2)) + input.solvation_beta) * np.exp((-denom/(2 * np.power(input.sigma, 2)))) / (np.power(input.sigma, 3))
            rec_solv += (4/3) * np.pi * np.power(rec.radii[a], 3) * np.exp((-denom/(2*np.power(input.sigma, 2)))) / (np.power(input.sigma, 3))
          
          denom = (d6 + input.deltaij6)
          vdwA += (4096*rec.epsilons_sqrt[a] * np.power(rec.radii[a], 6)) / np.power(denom, 2)
          vdwB += ( 128*rec.epsilons_sqrt[a] * np.power(rec.radii[a], 3)) / denom

        for m in range(len(rec.HBdonors)):
          d2 = grid.distance_squared(rec.xyz[rec.HBdonors[m][1]][0], x, rec.xyz[rec.HBdonors[m][1]][1], y, rec.xyz[rec.HBdonors[m][1]][2], z)
          d10 = np.double(np.power(d2, 5))
          ang = np.double(grid.angle(rec.xyz[rec.HBdonors[m][0]][0], rec.xyz[rec.HBdonors[m][0]][1], rec.xyz[rec.HBdonors[m][0]][2], 
                          rec.xyz[rec.HBdonors[m][1]][0], rec.xyz[rec.HBdonors[m][1]][1], rec.xyz[rec.HBdonors[m][1]][2], x, y, z))
          angle_term = np.double(np.power(np.cos(ang * np.pi / 180), 4))
          hb_donor += ((HB_C12/(d10*d2)) - (HB_C10/d10)) * angle_term

        for n in range(len(rec.HBacceptors)):
          d2 = grid.distance_squared(rec.xyz[rec.HBacceptors[n]][0], x, rec.xyz[rec.HBacceptors[n]][1], y, rec.xyz[rec.HBacceptors[n]][2], z)
          d10 = np.double(np.power(d2, 5))
          hb_acceptor += ((HB_C12/(d10*d2)) - (HB_C10/d10))

        grid.elec_grid[i][j][k] = elec
        grid.vdwA_grid[i][j][k] = vdwA
        grid.vdwB_grid[i][j][k] = vdwB
        grid.solv_gauss[i][j][k] = solv
        grid.rec_solv_gauss[i][j][k] = rec_solv
        grid.hb_donor_grid[i][j][k] = hb_donor
        grid.hb_acceptor_grid[i][j][k] = hb_acceptor;

  grid.rec_si = 0.0
  for a in range(rec.N):
    grid.rec_si += (input.solvation_alpha * np.power(rec.charges[a], 2)) + input.solvation_beta


In [6]:
@jit(nopython = True)
def distance_squared(x1, x2, y1, y2, z1, z2):
  return np.power(x2 - x1, 2) + np.power(y2 - y1, 2) + np.power(z2 - z1, 2)

In [7]:
@jit(nopython = True)
def angle(x1, y1, z1, x2, y2, z2, x3, y3, z3):
  ab = np.sqrt(distance_squared(x1, x2, y1, y2, z1, z2))
  ac = np.sqrt(distance_squared(x1, x3, y1, y3, z1, z3))
  bc = np.sqrt(distance_squared(x2, x3, y2, y3, z2, z3))

  angle = np.arccos((np.power(ab, 2) + np.power(bc, 2) - np.power(ac, 2))/(2*ab*bc))
  return angle * 180 / np.pi

In [ ]:
#testing
print(refgrid.distance_squared(1, 2, 3, 4, 5, 6))
print(distance_squared(1, 2, 3, 4, 5, 6))

In [ ]:
#testing

print(refgrid.angle(0, 0, 0, 4, 5, 6, 7, 8, 9))
print(angle(0, 0, 0, 4 ,5 ,6 ,7 ,8 ,9))
#angle.inspect_types()

High level objects to be ported:
- [x] Grid :: npointsx, npointsy, npointsz 
- [x] Grid :: grid_spacing
- [x] Grid :: xbegin, ybegin, zbegin 
- [x] Grid :: distance_squared() — faz inline
- [x] Grid :: angle()
- [x] Parser :: dieletric_model
- [x] Parser :: deltaij_es6
- [x] Parser :: solvation_alpha, salvation_beta
- [x] Parser :: sigma
- [x] Parser :: deltaij6
- [x] Mols :: N
- [x] Mol2 :: xyz (array)
- [x] Mol2 :: charges (array)
- [x] Mol2 :: radii (array)
- [x] Mol2 :: epsilons_sqrt (array)
- [x] Mol2 :: HBdonors (array)
- [x] Mol2 :: HBacceptors (array)
- [x] Outputs…




In [53]:
#@title (NUMBA) _compute_softcore_HB_omp_numba

#from numba.core.errors import NumbaDeprecationWarning, NumbaPendingDeprecationWarning
#import warnings

#warnings.simplefilter('ignore', category=NumbaDeprecationWarning)
#warnings.simplefilter('ignore', category=NumbaPendingDeprecationWarning)


@jit(nopython=True)
def _compute_grid_softcore_HB_omp_numba(npointsx, npointsy, npointsz,  # from Grid here onwards
                                   grid_spacing, 
                                   xbegin, ybegin, zbegin,
                                   dielectric_model, # from Parser here onwards 
                                   deltaij_es6, deltaij6,
                                   solvation_alpha, solvation_beta,
                                   sigma, diel,
                                   N, # from Mol2 here onwards
                                   xyz = np.empty((0,0), dtype=np.float64), 
                                   charges = np.empty(0, dtype=np.float64), 
                                   radii = np.empty(0, dtype=np.float64), 
                                   epsilons_sqrt = np.empty(0, dtype=np.float64), 
                                   HBdonors = np.empty((0,0), dtype=np.int64), 
                                   HBacceptors = np.empty(0, dtype=np.int64), 
                                   out_elec_grid = np.empty((0,0,0), dtype=np.float64), # output arrays from here onwards 
                                   out_vdwA_grid = np.empty((0,0,0), dtype=np.float64),
                                   out_vdwB_grid = np.empty((0,0,0), dtype=np.float64),
                                   out_solv_gauss = np.empty((0,0,0), dtype=np.float64),
                                   out_rec_solv_gauss = np.empty((0,0,0), dtype=np.float64),
                                   out_hb_donor_grid = np.empty((0,0,0), dtype=np.float64),
                                   out_hb_acceptor_grid = np.empty((0,0,0), dtype=np.float64)):
  
  #if input.scoring_function >= 2:
  #  compute_grid_hardcore_HB_omp(...)
  #  return

  HB_C12 = np.double(55332.873)
  HB_C10 = np.double(18393.199)

  for i in range(npointsx):
    x = np.double(i*grid_spacing + xbegin) 

    for j in range(npointsy):
      y = np.double(j*grid_spacing + ybegin) 

      for k in range(npointsz):
        z = np.double(k*grid_spacing + zbegin) 

        elec = np.double(0.0)
        vdwA = np.double(0.0)
        vdwB = np.double(0.0) 
        solv = np.double(0.0) 
        rec_solv = np.double(0.0) 
        hb_donor = np.double(0.0) 
        hb_acceptor = np.double(0.0) 

        for a in range(N):
          
          d2 = np.double(distance_squared(x, xyz[a][0], y, xyz[a][1], z, xyz[a][2]))
          d6 = np.double(np.power(d2, 3))
          
          denom = np.double(0.0)
            
          if dielectric_model == "constant":
            denom = np.power(d6 + deltaij_es6, 1/3)
            elec += (332.0*charges[a])/(diel*denom)
            solv += ((solvation_alpha * charges[a] * charges[a]) + solvation_beta) * np.exp((-denom/(2 * np.power(sigma, 2)))) / (np.power(sigma, 3))
            rec_solv += (4/3) * np.pi * np.power(radii[a], 3) * np.exp((-denom/(2*np.power(sigma, 2)))) / (np.power(sigma, 3))

          else:
            denom = np.power(d6 + deltaij_es6, 1/3)
            elec += (332.0*charges[a]/(diel*denom))
            solv += ((solvation_alpha * np.power(charges[a], 2)) + solvation_beta) * np.exp((-denom/(2 * np.power(sigma, 2)))) / (np.power(sigma, 3))
            rec_solv += (4/3) * np.pi * np.power(radii[a], 3) * np.exp((-denom/(2*np.power(sigma, 2)))) / (np.power(sigma, 3))
          
          denom = (d6 + deltaij6)
          vdwA += (4096*epsilons_sqrt[a] * np.power(radii[a], 6)) / np.power(denom, 2)
          vdwB += ( 128*epsilons_sqrt[a] * np.power(radii[a], 3)) / denom

        for m in range(len(HBdonors)):
          HBd0 = HBdonors[m][0]
          HBd1 = HBdonors[m][1]
          d2 = distance_squared(xyz[HBd1][0], x, xyz[HBd1][0], y, xyz[HBd1][0], z)
          d10 = np.double(np.power(d2, 5))
          ang = np.double(angle(xyz[HBdonors[m][0]][0], xyz[HBdonors[m][0]][1], xyz[HBdonors[m][0]][2], 
                          xyz[HBdonors[m][1]][0], xyz[HBdonors[m][1]][1], xyz[HBdonors[m][1]][2], x, y, z))
          angle_term = np.double(np.power(np.cos(ang * np.pi / 180), 4))
          hb_donor += ((HB_C12/(d10*d2)) - (HB_C10/d10)) * angle_term

        for n in range(len(HBacceptors)):
          d2 = distance_squared(xyz[HBacceptors[n]][0], x, xyz[HBacceptors[n]][1], y, xyz[HBacceptors[n]][2], z)
          d10 = np.double(np.power(d2, 5))
          hb_acceptor += ((HB_C12/(d10*d2)) - (HB_C10/d10))

        out_elec_grid[i][j][k] = elec
        out_vdwA_grid[i][j][k] = vdwA
        out_vdwB_grid[i][j][k] = vdwB
        out_solv_gauss[i][j][k] = solv
        out_rec_solv_gauss[i][j][k] = rec_solv
        out_hb_donor_grid[i][j][k] = hb_donor
        out_hb_acceptor_grid[i][j][k] = hb_acceptor;

  #grid.rec_si = 0.0
  #for a in range(rec.N):
  #  grid.rec_si += (input.solvation_alpha * np.power(rec.charges[a], 2)) + input.solvation_beta


In [54]:
#@title (Wrapper) compute_softcore_HB_omp_numba
def compute_grid_softcore_HB_omp_numba(rec, grid, input):
  """Wrapper for calling the njit function"""

  out_elec_grid = np.empty((grid.npointsx, grid.npointsy, grid.npointsz))
  out_vdwA_grid = np.empty((grid.npointsx, grid.npointsy, grid.npointsz))
  out_vdwB_grid = np.empty((grid.npointsx, grid.npointsy, grid.npointsz))
  out_solv_gauss = np.empty((grid.npointsx, grid.npointsy, grid.npointsz))
  out_rec_solv_gauss = np.empty((grid.npointsx, grid.npointsy, grid.npointsz))
  out_hb_donor_grid = np.empty((grid.npointsx, grid.npointsy, grid.npointsz))
  out_hb_acceptor_grid = np.empty((grid.npointsx, grid.npointsy, grid.npointsz))

  HBdonors = np.array(rec.HBdonors)
  if not np.size(HBdonors):
    HBdonors = np.empty((0,0), dtype=np.int64)

  _compute_grid_softcore_HB_omp_numba(grid.npointsx, grid.npointsy, grid.npointsz, 
                                  grid.grid_spacing,
                                  grid.xbegin, grid.ybegin, grid.zbegin,
                                  input.dielectric_model,
                                  input.deltaij_es6, input.deltaij6,
                                  input.solvation_alpha, input.solvation_beta,
                                  input.sigma, input.diel,
                                  rec.N,
                                  np.array(rec.xyz),
                                  np.array(rec.charges),
                                  np.array(rec.radii),
                                  np.array(rec.epsilons_sqrt),
                                  HBdonors,
                                  np.array(rec.HBacceptors, dtype=np.int64),
                                  out_elec_grid,
                                  out_vdwA_grid,
                                  out_vdwB_grid,
                                  out_solv_gauss,
                                  out_rec_solv_gauss,
                                  out_hb_donor_grid,
                                  out_hb_acceptor_grid)
  
  # TODO: think about the best way of doing this copy operation
  for i in range(grid.npointsx):
    for j in range(grid.npointsy):
      for k in range(grid.npointsz):
        grid.elec_grid[i][j][k] = out_elec_grid[i][j][k]
        grid.vdwA_grid[i][j][k] = out_vdwA_grid[i][j][k]
        grid.vdwB_grid[i][j][k] = out_vdwB_grid[i][j][k]
        grid.solv_gauss[i][j][k] = out_solv_gauss[i][j][k]
        grid.rec_solv_gauss[i][j][k] = out_rec_solv_gauss[i][j][k]
        grid.hb_donor_grid[i][j][k] = out_hb_donor_grid[i][j][k]
        grid.hb_acceptor_grid[i][j][k] = out_hb_acceptor_grid[i][j][k]
  



In [ ]:
arr = np.array(rec.xyz)
print(arr)

In [ ]:
compute_grid_softcore_HB_omp(rec, grid, Input)

100%|██████████| 8/8 [01:12<00:00,  9.01s/it]


In [ ]:
compute_grid_softcore_HB_numba(rec, grid, Input)

#Print grids

In [ ]:
for i in range(grid.npointsx):
  print('\n------------------------')
  for j in range(grid.npointsy):
    print('\n')
    for k in range(grid.npointsz):
      print(f'{grid.elec_grid[i][j][k]:.2f}', end=' ')

In [ ]:
for i in range(grid.npointsx):
  print('\n------------------------')
  for j in range(grid.npointsy):
    print('\n')
    for k in range(grid.npointsz):
      print(f'{refgrid.elec_grid[i][j][k]:.2f}', end=' ')

#Benchmarking

In [ ]:
# search_box_x == search_box_y == search_box_z == 30.0
# x_dim == y_dim == z_dim == 30.0 
# grid_spacing == 1.0

%timeit refgrid.compute_grid_hardcore_HB_omp(rec) # versão C++
%timeit compute_grid_softcore_HB_omp(rec, py_grid, Input) # versão python usando objetos expostos pelo boost
%timeit compute_grid_softcore_HB_omp_numba(rec, numba_grid, Input) # versão numba usando njit

9.38 s ± 854 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


# Misc

In [ ]:
@jit
def foo(npointsx):
  for i in range(npointsx):
    n = i**2

In [ ]:
foo(grid.npointsx)
foo.inspect_types()

In [23]:
test_arr = np.size(np.array([1]))
bool(test_arr)

True